In [1]:
from ultralytics import YOLO
import cv2
import os
import glob
import os
import glob
import random
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from ultralytics import YOLO
import torch
from pathlib import Path

In [3]:
import os
print(os.getcwd())


/Users/sahilbodkhe/Desktop/Experiment4/SYDE770


In [2]:
# Paths
BASE_DIR = "/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups"  # Change to your dataset path
IMAGE_DIR = os.path.join(BASE_DIR, "images")
LABEL_DIR = os.path.join(BASE_DIR, "data")
OUTPUT_DIR = os.path.join(BASE_DIR, "yolo_dataset")
MODEL_PATH = "yolov5"  # Change if using another YOLOv5 model

# Create output directories
os.makedirs(os.path.join(OUTPUT_DIR, "train", "images"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "train", "labels"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "val", "images"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "val", "labels"), exist_ok=True)

# Load images & labels
image_paths = glob.glob(os.path.join(IMAGE_DIR, "*"))
image_list = []
labels = []

for img_path in image_paths:
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(LABEL_DIR, base_name + ".txt")

    if os.path.exists(label_path) and os.path.getsize(label_path) > 0:
        with open(label_path, "r") as file:
            label_data = file.readlines()
            class_ids = [int(line.split()[0]) for line in label_data]
            label_class = 1 if 1 in class_ids else 0  # If a Tim Hortons cup is present, label as 1
    else:
        label_class = 0  # No annotation means it's a normal cup (background class)

    image_list.append((img_path, label_path))
    labels.append(label_class)

# Convert to DataFrame for processing
df = pd.DataFrame(image_list, columns=["image", "label"])
df["class"] = labels

# Print dataset summary
print(f"Total Images: {len(df)}")
print(f"Class Distribution: {df['class'].value_counts()}")


Total Images: 1669
Class Distribution: class
1    1026
0     643
Name: count, dtype: int64


In [4]:
import shutil
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Create necessary directories and copy data (train/validation)
OUTPUT_DIR = "C:/Users/Chinmay Nagesh/Desktop/Yolo/SYDE770/Normal_cups/yolo_dataset"

# Stratified K-Fold Cross Validation (Keep Class Balance)
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Select one fold as validation (80-20 Split)
for train_idx, val_idx in skf.split(df["image"], df["class"]):
    train_data = df.iloc[train_idx]
    val_data = df.iloc[val_idx]
    break  # We only need the first split

# Function to copy files
def copy_files(data, split):
    for _, row in data.iterrows():
        shutil.copy(row["image"], os.path.join(OUTPUT_DIR, split, "images", os.path.basename(row["image"])))

        if os.path.exists(row["label"]) and os.path.getsize(row["label"]) > 0:
            shutil.copy(row["label"], os.path.join(OUTPUT_DIR, split, "labels", os.path.basename(row["label"])))
        else:
            # Create an empty label file if no annotation exists
            open(os.path.join(OUTPUT_DIR, split, "labels", os.path.basename(row["label"])), 'w').close()

copy_files(train_data, "train")
copy_files(val_data, "val")

# Create YOLO dataset.yaml file
yaml_content = f"""
path: C:/Users/Chinmay Nagesh/Desktop/Yolo/SYDE770/Normal_cups/yolo_dataset
train: train/images
val: val/images
nc: 2
names: ["Normal Cup", "Tim Hortons Cup"]
"""

yaml_path = "C:/Users/Chinmay Nagesh/Desktop/Yolo/SYDE770/Normal_cups/yolo_dataset/dataset.yaml"
with open(yaml_path, "w") as f:
    f.write(yaml_content)

# Train YOLOv5 (No MLflow integration in this cell)
command = r'python yolov5/train.py --imgsz 320 --batch-size 32 --epochs 5 --data "C:/Users/Chinmay Nagesh/Desktop/Yolo/SYDE770/Normal_cups/yolo_dataset/dataset.yaml" --weights yolov5s.pt --device cpu --freeze 10'
os.system(command)

print("Training completed! Model saved in YOLO output folder.")


Training completed! Model saved in YOLO output folder.


In [3]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Define constants
OUTPUT_DIR = "/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset"
TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
VAL_DIR = os.path.join(OUTPUT_DIR, "val")

# Ensure necessary directories exist
def create_dirs():
    for split in ["train", "val"]:
        os.makedirs(os.path.join(OUTPUT_DIR, split, "images"), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_DIR, split, "labels"), exist_ok=True)



# Stratified K-Fold Split
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

for train_idx, val_idx in skf.split(df["image"], df["class"]):
    train_data = df.iloc[train_idx]
    val_data = df.iloc[val_idx]
    break  # Only using the first split

# Function to copy files
def copy_files(data, split):
    for _, row in data.iterrows():
        img_src = row["image"]
        label_src = row["label"]
        
        img_dst = os.path.join(OUTPUT_DIR, split, "images", os.path.basename(img_src))
        label_dst = os.path.join(OUTPUT_DIR, split, "labels", os.path.basename(label_src))
        
        if os.path.exists(img_src):
            shutil.copy(img_src, img_dst)
        else:
            print(f"Warning: Image file not found - {img_src}")
        
        if os.path.exists(label_src) and os.path.getsize(label_src) > 0:
            shutil.copy(label_src, label_dst)
        else:
            open(label_dst, 'w').close()  # Create empty label if missing

# Create required directories
create_dirs()
copy_files(train_data, "train")
copy_files(val_data, "val")

# Generate dataset.yaml file
yaml_content = f"""
path: {OUTPUT_DIR}
train: train/images
val: val/images
nc: 2
names: ["Normal Cup", "Tim Hortons Cup"]
"""

yaml_path = os.path.join(OUTPUT_DIR, "dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

# Train YOLOv5
command = (
    f'python3 yolov5/train.py --imgsz 320 --batch-size 32 --epochs 5 '
    f'--data "{yaml_path}" --weights yolov5s.pt --device mps --freeze 10'
)

exit_code = os.system(command)
if exit_code != 0:
    print("Error: YOLO training failed. Check the logs for details.")
else:
    print("Training completed! Model saved in YOLO output folder.")

train: weights=yolov5s.pt, cfg=, data=/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=32, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=mps, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-398-g5cdad892 Python-3.11.9 torch-2.6.0 MPS

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_mom

Error decoding JSON from /Users/sahilbodkhe/Library/Application Support/Ultralytics/settings.json. Starting with an empty dictionary.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/sahilbodkhe/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


train: Scanning /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/labels... 1335 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1335/1335 [00:04<00:00, 327.28it/s]
train: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/images/GenAI_Img_4_2.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/images/IMG20250219140934.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/images/IMG20250219140944.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/images/IMG20250219141012.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/images/IMG20250219141038.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Use

Error decoding JSON from /Users/sahilbodkhe/Library/Application Support/Ultralytics/settings.json. Starting with an empty dictionary.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/sahilbodkhe/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Error decoding JSON from /Users/sahilbodkhe/Library/Application Support/Ultralytics/persistent_cache.json. Starting with an empty dictionary.


val: Scanning /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/val/labels... 334 images, 0 backgrounds, 0 corrupt: 100%|██████████| 334/334 [00:09<00:00, 33.55it/s]
val: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/val/images/IMG20250219140849.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/val/images/IMG20250219141339.jpg: corrupt JPEG restored and saved
val: New cache created: /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/val/labels.cache

AutoAnchor: 5.29 anchors/target, 1.000 Best Possible Recall (BPR). Current anchors are a good fit to dataset ✅
Plotting labels to yolov5/runs/train/exp3/labels.jpg... 
/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/yolov5/train.py:355: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.G

Training completed! Model saved in YOLO output folder.


Keeping results ready for MLflow

In [15]:
import numpy
import ultralytics
print(numpy.__version__)  # Should print 2.1.1
print(ultralytics.__version__)  # Should match 8.3.88


2.1.1
8.3.88


In [4]:
import pandas as pd
import os

# Path to results.csv in latest exp folder
latest_exp= 'yolov5/runs/train/exp3'
results_path = os.path.join(latest_exp, "results.csv")

# Check if results.csv exists
if os.path.exists(results_path):
    df = pd.read_csv(results_path)
    # print("Available columns in results.csv:", df.columns.tolist())  # Print column names
    df.columns = df.columns.str.strip()
    # print(df.iloc[-1])  # See exact column names

train_box_loss = df.iloc[-1]["train/box_loss"]
train_obj_loss=df.iloc[-1]["train/obj_loss"]
train_cls_loss=df.iloc[-1]["train/cls_loss"]
val_box_loss=df.iloc[-1]["val/box_loss"]
mAP_50=df.iloc[-1]["metrics/mAP_0.5"]
mAP_50_95=df.iloc[-1]["metrics/mAP_0.5:0.95"]
precision=df.iloc[-1]["metrics/precision"]
recall=df.iloc[-1]["metrics/recall"]

ML Flow section

In [5]:
import mlflow
import dagshub
import pandas as pd
import shutil
import os

# Initialize MLflow tracking
dagshub.init(repo_owner='chinmay-nagesh',
             repo_name='SYDE770-dagshub',
             mlflow=True)

mlflow.set_tracking_uri("https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow")
mlflow.set_experiment("YOLOv5 Training")

# Start MLflow run to track hyperparameters and metrics
with mlflow.start_run():
    # Log Hyperparameters
    mlflow.log_param("img_size", 320)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("epochs", 5)
    mlflow.log_param("weights", "yolov5s.pt")
    mlflow.log_param("device", "cpu")
    mlflow.log_param("freeze", 10)
    

    # Log Metrics
    mlflow.log_metric("train_box_loss", train_box_loss)
    mlflow.log_metric("train_obj_loss", train_obj_loss)
    mlflow.log_metric("train_cls_loss", train_cls_loss)
    mlflow.log_metric("val_box_loss", val_box_loss)
    mlflow.log_metric("mAP_50", mAP_50)
    mlflow.log_metric("mAP_50_95", mAP_50_95)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)

    # Log Model
    best_model_path = "yolov5/runs/train/exp/weights/best.pt"
    model_save_path = "yolov5_model"

    if os.path.exists(best_model_path):
        os.makedirs(model_save_path, exist_ok=True)
        shutil.copy(best_model_path, os.path.join(model_save_path, "best.pt"))

        # Log Model as an artifact in MLflow
        mlflow.log_artifact(os.path.join(model_save_path, "best.pt"), artifact_path="models/yolov5")

        # Check if model "yolov5" exists in MLflow model registry
        client = mlflow.tracking.MlflowClient()
        model_name = "yolov5"

        try:
            # Try registering a new version of the existing model
            mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/models/yolov5/best.pt", model_name)
            print(f"New version of {model_name} registered in MLflow.")
        except Exception:
            # If model does not exist, create a new registered model
            mlflow.create_registered_model(model_name)
            mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/models/yolov5/best.pt", model_name)
            print(f"Model {model_name} created and registered in MLflow.")

    print("Metrics and model logged to MLflow.")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=46bc746d-70c3-4d67-81e7-8da7e7067fce&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ec6b43d9757d4f5bac8e151aa46e6888dab2aeb74e1b2b4115c1bcab4ccf3dea




Accessing as SAHILBODKHE

Initialized MLflow to track repo "chinmay-nagesh/SYDE770-dagshub"

Repository chinmay-nagesh/SYDE770-dagshub initialized!

Metrics and model logged to MLflow.
🏃 View run fearless-cat-924 at: https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow/#/experiments/1/runs/020ca79ae9f24e208e4a8491445f4cf1
🧪 View experiment at: https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow/#/experiments/1
